In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-03-29 15:44:11.936088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df = pd.read_csv('../datasets/churn.csv', index_col='RowNumber')

In [21]:
df.drop(columns=['CustomerId', 'Surname'], inplace=True)

In [22]:
df.describe()[1:]

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
mean,650.53,38.92,5.01,"76,485.89",1.53,0.71,0.52,"100,090.24",0.20
std,96.65,10.49,2.89,"62,397.41",0.58,0.46,0.50,"57,510.49",0.40
min,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.00
25%,584.00,32.00,3.00,0.00,1.00,0.00,0.00,"51,002.11",0.00
50%,652.00,37.00,5.00,"97,198.54",1.00,1.00,1.00,"100,193.91",0.00
75%,718.00,44.00,7.00,"127,644.24",2.00,1.00,1.00,"149,388.25",0.00
max,850.00,92.00,10.00,"250,898.09",4.00,1.00,1.00,"199,992.48",1.00


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 937.5+ KB


In [25]:
df.Geography.unique(), df.Gender.unique()

(array(['France', 'Spain', 'Germany'], dtype=object),
 array(['Female', 'Male'], dtype=object))

In [26]:
df.Geography = pd.Categorical(df.Geography)
df.Gender = pd.Categorical(df.Gender)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      10000 non-null  int64   
 1   Geography        10000 non-null  category
 2   Gender           10000 non-null  category
 3   Age              10000 non-null  int64   
 4   Tenure           10000 non-null  int64   
 5   Balance          10000 non-null  float64 
 6   NumOfProducts    10000 non-null  int64   
 7   HasCrCard        10000 non-null  int64   
 8   IsActiveMember   10000 non-null  int64   
 9   EstimatedSalary  10000 non-null  float64 
 10  Exited           10000 non-null  int64   
dtypes: category(2), float64(2), int64(7)
memory usage: 801.0 KB


In [38]:
df = df[['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
                  'Geography', 'Gender', 'Exited']]

In [32]:
df.head(1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography,Gender,Exited
RowNumber,,,,,,,,,,,
1,619,42,2,0.00,1,1,1,"101,348.88",France,Female,1


In [49]:
for col in ['Age', 'Tenure', 'HasCrCard', 'HasCrCard', 'NumOfProducts', 'IsActiveMember', 'Exited']:
    df[col] = df[col].astype('uint8')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      10000 non-null  int64   
 1   Age              10000 non-null  uint8   
 2   Tenure           10000 non-null  uint8   
 3   Balance          10000 non-null  float64 
 4   EstimatedSalary  10000 non-null  float64 
 5   NumOfProducts    10000 non-null  uint8   
 6   HasCrCard        10000 non-null  uint8   
 7   IsActiveMember   10000 non-null  uint8   
 8   Geography        10000 non-null  category
 9   Gender           10000 non-null  category
 10  Exited           10000 non-null  uint8   
dtypes: category(2), float64(2), int64(1), uint8(6)
memory usage: 390.9 KB


In [46]:
np.iinfo('uint8')

iinfo(min=0, max=255, dtype=uint8)

In [54]:
df.iloc[:, :-1].head(1)

,CreditScore,Age,Tenure,Balance,EstimatedSalary,NumOfProducts,HasCrCard,IsActiveMember,Geography,Gender
RowNumber,,,,,,,,,,
1,619,42,2,0.00,"101,348.88",1,1,1,France,Female


In [52]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [53]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [11]:
X[:2]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58]],
      dtype=object)

Split the data

In [56]:
import sklearn

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2912)

In [57]:
sklearn.__version__

'1.2.2'